## Simulation as Optimization: reuse code

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch, time

from celluloid import Camera
from IPython.display import HTML
from base64 import b64encode


def make_video(xs, path, interval=60, ms=10, **kwargs): # xs: [time, N, 2]
    fig = plt.gcf() ; fig.set_dpi(100) ; fig.set_size_inches(3, 3)
    camera = Camera(fig)
    for i in range(xs.shape[0]):
        plt.plot(xs[i][...,0], xs[i][...,1], 'k.', markersize=ms)
        plt.axis('equal') ; plt.xlim(0,1) ; plt.ylim(0,1)
        plt.xticks([], []); plt.yticks([], [])
        camera.snap()
    anim = camera.animate(blit=True, interval=interval, **kwargs)
    anim.save(path) ; plt.close()

## Get a baseline simulation working

In [ ]:
def V_gas(xs, eps=1e-6, overlap_radius=0.05, scale_coeff=1e-5): # 1e-6 -> 500 particles
    if len(xs.shape) > 2:
        return sum([V_gas(_xs, overlap_radius, scale_coeff) for _xs in xs]) # broadcast
    else:
        dist_matrix = ((xs[:,0:1] - xs[:,0:1].T).pow(2) + (xs[:,1:2] - xs[:,1:2].T).pow(2) + eps).sqrt()
        dists = dist_matrix[torch.triu_indices(xs.shape[0], xs.shape[0], 1).split(1)]
        # potentials  = (dists > 1-overlap_radius) * (5e2*(overlap_radius - (1-dists)) + 1/overlap_radius) # cap
        potentials = (dists > 0.5) * (dists < 1-overlap_radius) * 1/(1-dists + eps)  # 1/(1-r) (wraparound)
        potentials += (dists > overlap_radius)* (dists < 0.5) * 1/(dists + eps)  # 1/r
        potentials += (dists < overlap_radius) * (5e2*(overlap_radius - dists) + 1/overlap_radius)  # cap
        return potentials.sum() * scale_coeff

def V_3body(xs, eps=1e-6, overlap_radius=0.05, scale_coeff=1.3e-4):
    if len(xs.shape) > 2:
        return sum([V_3body(_xs, overlap_radius, scale_coeff) for _xs in xs]) # broadcast
    else:
        dist_matrix = ((xs[:,0:1] - xs[:,0:1].T).pow(2) + (xs[:,1:2] - xs[:,1:2].T).pow(2) + eps).sqrt()
        dists = dist_matrix[torch.triu_indices(xs.shape[0], xs.shape[0], 1).split(1)]
        potentials =  (dists > overlap_radius) * 1/(dists + eps)  # 1/r^2
        potentials += (dists < overlap_radius) * (5e2*(overlap_radius - dists) + 1/overlap_radius)
        return -potentials.sum() * scale_coeff

def V_freebody(xs): # assume xs are measured along vertical axis
    return xs.sum()

def V_pend(x, m=1, l=150, g=1):
    return -m*l*g*(torch.cos(x) - 1)

def T_pend(xdot, m=1, l=150, g=1):
    return m*l*g*(l*xdot**2) / (2 * g)

def V_dblpend(x, m1=1, m2=1, l1=1, l2=1, g=1):
    th1, th2 = x[...,0], x[...,1]
    return -(m1 + m2) * l1 * g * torch.cos(th1) - m2 * l2 * g * torch.cos(th2)
    
def T_dblpend(x, xdot, m1=1, m2=1, l1=1, l2=1, g=1):
    th1, th2 = x[...,0], x[...,1]
    th1d, th2d = xdot[...,0], xdot[...,1]
    return 0.5 * m1 * (l1 * th1d) ** 2 + 0.5 * m2 * ((l1 * th1d) ** 2 + (l2 * th2d) ** 2 +
                                                     2 * l1 * l2 * th1d * th2d * torch.cos(th1 - th2))

def forces(xs, xdot=None, potential_fn=None, **kwargs):
    xs.requires_grad = True
    return -torch.autograd.grad(potential_fn(xs), xs)[0]

## Simulation code

In [ ]:
def solve_ode_euler(x0, x1, dt, forces_fn, steps=100, box_width=1):
    xs = [x0, x1]
    ts = [0, dt]
    xdot = (x1 - x0) / dt
    x = xs[-1]
    for i in range(steps-2):
        a = forces_fn(x, xdot)
        xdot = xdot + a*dt
        x = x + xdot*dt
        xs.append(x)
        ts.append(ts[-1]+dt)
    return np.asarray(ts), np.stack(xs)

def simulate_freebody(dt=0.25, steps=60):
    np.random.seed(1)
    x0, x1 = np.asarray([0.]), np.asarray([2.])
    v0 = (x1 - x0) / dt
    forces_fn = lambda x, xdot: forces(torch.tensor(x), None, potential_fn=V_freebody).numpy()
    return solve_ode_euler(x0, x1, dt, forces_fn, steps=steps, box_width=100)

def simulate_pend(dt=1):
    np.random.seed(1)
    x0, x1 = np.asarray([np.pi/2.]), np.asarray([np.pi/2.])
    def pend_forces_fn(x, xdot, g=1, l=150):
        return -g / l * np.sin(x)
    return solve_ode_euler(x0, x1, dt, pend_forces_fn)


#### DOUBLE PENDULUM ####

def dblpend_forces_fn(x, xdot, m1=1, m2=1, l1=1, l2=1, g=1):
    """Return the first derivatives of x and xdot"""
    theta1, theta2 = x
    theta1dot, theta2dot = xdot
    c, s = np.cos(theta1-theta2), np.sin(theta1-theta2)

    z1dot = (m2*g*np.sin(theta2)*c - m2*s*(l1*theta1dot**2*c + l2*theta2dot**2) -
             (m1+m2)*g*np.sin(theta1)) / l1 / (m1 + m2*s**2)
    z2dot = ((m1+m2)*(l1*theta1dot**2*s - g*np.sin(theta2) + g*np.sin(theta1)*c) + 
             m2*l2*theta2dot**2*s*c) / l2 / (m1 + m2*s**2)
    return np.asarray([z1dot, z2dot])

def radial2cartesian(thetas, l1=1, l2=1): # Convert from radial to Cartesian coordinates.
    t1, t2 = thetas.T
    x1 = l1 * np.sin(t1)
    y1 = -l1 * np.cos(t1)
    x2 = x1 + l2 * np.sin(t2)
    y2 = y1 - l2 * np.cos(t2)
    return np.stack([x1, y1, x2, y2]).T.reshape(-1,2,2)/5 + 0.6

def simulate_dblpend(dt=1):
    np.random.seed(1)
    x0 = np.asarray([3*np.pi/7, 3*np.pi/4]) ; x1 = np.copy(x0)
    return solve_ode_euler(x0, x1, dt, dblpend_forces_fn)


#### 3 BODY AND GAS SIMULATIONS ####

def simulate_3body(dt=0.5, steps=100):
    np.random.seed(1)
    x0 = np.asarray([[0.4, 0.3], [0.4, 0.7], [0.7, 0.5]])
    v0 = np.asarray([[0.017, -0.006], [-.012, -.012], [0.0, 0.017]])
    x1 = x0 + dt*v0
    forces_fn = lambda x, xdot: forces(torch.tensor(x), None, potential_fn=V_3body).numpy()
    return solve_ode_euler(x0, x1, dt, forces_fn, steps=steps)

def simulate_gas(dt=1, N=50):
    np.random.seed(1)
    x0 = np.random.rand(N,2)*.8 + 0.1
    v0 = np.random.randn(N,2)*.00
    x1 = x0 + dt*v0
    forces_fn = lambda x, xdot: forces(torch.tensor(x), None, potential_fn=V_gas).numpy()
    return solve_ode_euler(x0, x1, dt, forces_fn)

In [ ]:
t_sim, x_sim = simulate_dblpend(dt=0.06)
plt.title('Double pendulum')
plt.plot(t_sim, x_sim, 'k.-')
plt.show()

make_video(radial2cartesian(x_sim), path='sim.mp4', interval=60, ms=20)
mp4 = open('sim.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML('<video width=300 controls><source src="{}" type="video/mp4"></video>'.format(data_url))

In [ ]:
t_sim, x_sim = simulate_freebody(dt=0.25, steps=60)
plt.title('Particle in free fall')
plt.plot(t_sim, x_sim, 'k.-')
plt.show()

In [ ]:
t_sim, x_sim = simulate_pend(dt=1)
plt.title('Pendulum')
plt.plot(t_sim, x_sim, 'k.-')
plt.show()

In [ ]:
t, x = simulate_3body()
make_video(x, path='sim.mp4', interval=60, ms=20)
mp4 = open('sim.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML('<video width=300 controls><source src="{}" type="video/mp4"></video>'.format(data_url))

In [ ]:
t, x = simulate_gas(dt=.5, N=50)

make_video(x, path='sim.mp4', interval=30)
mp4 = open('sim.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML('<video width=300 controls><source src="{}" type="video/mp4"></video>'.format(data_url))

## Recover the same dynamics by minimizing the action

In [ ]:
def lagrangian_pend(x, xdot, m=1):
    norm_factor = x.shape[0]
    return (T_pend(xdot).sum() - V_pend(x).sum()) / norm_factor

def lagrangian_dblpend(x, xdot, m=1):
    norm_factor = x.shape[0]
    return (T_dblpend(x, xdot).sum() - V_dblpend(x).sum()) / norm_factor

def lagrangian_freebody(x, xdot, m=1):
    norm_factor = x.shape[0]
    return ((.5*m*xdot**2).sum() - V_freebody(x.reshape(-1, 1)).sum()) / norm_factor

def lagrangian_3body(x, xdot, m=1):
    N = x.shape[-1] // 2
    norm_factor = x.shape[0]*N
    return ((.5*m*xdot**2).sum() - V_3body(x.reshape(-1, N, 2)).sum()) / norm_factor

def lagrangian_gas(x, xdot, m=1):
    N = x.shape[-1] // 2
    norm_factor = x.shape[0]*N
    return ((.5*m*xdot**2).sum() -V_gas(x.reshape(-1, N, 2)).sum()) / norm_factor
  
def action(x, L_fn, dt):
    xdot = (x[1:] - x[:-1]) / dt
    xdot = torch.cat([xdot, xdot[-1:]], axis=0)
    return L_fn(x, xdot).sum()

def get_path_between(path, steps, step_size, L_fn, dt, opt='sgd', print_every=15):
    t = np.linspace(0, len(path.x)-1, len(path.x)) * dt
    optimizer = torch.optim.SGD(path.parameters(), lr=step_size, momentum=0) if opt=='sgd' else \
                torch.optim.Adam(path.parameters(), lr=step_size)
    xs = [path.x.clone().data]
    t0 = time.time()
    for i in range(steps):
        S = action(path.x, L_fn, dt)
        S.backward() ; path.x.grad.data[[0,-1]] *= 0
        optimizer.step() ; path.zero_grad()

        if i % (steps//print_every) == 0:
            xs.append(path.x.clone().data)
            print('step={:04d}, S={:.3e} J*s, dt={:.1f}s'.format(i, S.item(), time.time()-t0))
            t0 = time.time()
    return t, path, xs

class PerturbedPath(torch.nn.Module):
    def __init__(self, x_true, N, sigma=0, shift=False, zero_basepath=False, coords=2):
        super(PerturbedPath, self).__init__()
        np.random.seed(0)
        self.x_true = x_true
        x_noise = sigma*np.random.randn(*x_true.shape).clip(-1,1)
        x_noise[:1] = x_noise[-1:] = 0
        x_basepath = np.copy(x_true)
        x_basepath[1:-1] = x_basepath[1:-1]*0 if zero_basepath else x_basepath[1:-1]
        self.x_pert = x_pert = (x_basepath + x_noise).reshape(-1, N*coords)
        if shift:
            x_pert_shift = np.concatenate([x_pert[5:-5,2:], x_pert[5:-5,:2]], axis=-1)
            self.x_pert[5:-5] = x_pert[5:-5] = x_pert_shift
            print(self.x_pert.shape)
        self.x = torch.nn.Parameter(torch.tensor(x_pert)) # [time, N*2]

## Double pendulum

In [ ]:
dt = 0.06 ; N = 2
t_sim, x_sim = simulate_dblpend(dt=dt)

init_path = PerturbedPath(x_sim, N=N, coords=1, sigma=1e0, zero_basepath=False) # [time, N*2]
t_min, path, xs_min = get_path_between(init_path, steps=100, step_size=1e-1, 
                                       L_fn=lagrangian_dblpend, dt=dt, opt='adam')

In [ ]:
plt.title('Double Pendulum (theta 2)')
plt.plot(t_sim, x_sim[:,1], 'k.-', label='Simulator')
plt.plot(t_min, xs_min[0][:,1], 'y-', label='Initial path')
plt.plot(t_min, xs_min[-1][:,1], 'b.-', label='Path of least action')
plt.legend()
plt.tight_layout() ; plt.show()

In [ ]:
make_video(radial2cartesian(xs_min[-1]), path='sim.mp4', interval=60, ms=20)
mp4 = open('sim.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML('<video width=300 controls><source src="{}" type="video/mp4"></video>'.format(data_url))

## Free body

In [ ]:
dt = 0.25 ; N = 1
t_sim, x_sim = simulate_freebody(dt=dt, steps=60)
init_path = PerturbedPath(x_sim, N=N, coords=1, sigma=1.5e0, zero_basepath=True) # [time, N*2]
t_min, path, xs_min = get_path_between(init_path, steps=500, step_size=1e0, 
                                       L_fn=lagrangian_freebody, dt=dt, opt='adam')

In [ ]:
plt.figure(dpi=90)
plt.title('Particle in freefall')
plt.plot(t_sim, x_sim, 'r-', label='ODE solution')

plt.plot(t_min, xs_min[0], 'y.-', label='Initial (random) path')
for i, xi in enumerate(xs_min):
    label = 'During optimization' if i==10 else None
    plt.plot(t_min, xi, alpha=0.3, color=plt.cm.viridis( 1-i/(len(xs_min)-1) ), label=label)
plt.plot(t_min, xs_min[-1], 'b.-', label='Final (optimized) path')
plt.plot(t_min[[0,-1]], xs_min[0].data[[0,-1]], 'b+', markersize=15, label='Points held constant')

plt.ylim(-5, 40)
plt.xlabel('Time (s)') ; plt.ylabel('Height (m)') ; plt.legend(fontsize=8, ncol=3)
plt.tight_layout() ; plt.show()

## Pendulum

In [ ]:
dt = 1 ; N = 1
t_sim, x_sim = simulate_pend(dt=dt)

init_path = PerturbedPath(x_sim, N=N, coords=1, sigma=1.5e0, zero_basepath=True) # [time, N*2]
t_min, path, xs_min = get_path_between(init_path, steps=1000, step_size=1e0, 
                                       L_fn=lagrangian_pend, dt=dt, opt='adam')

plt.title('Pendulum')
plt.plot(t_sim, x_sim, 'k.-', label='Simulator')
plt.plot(t_min, xs_min[-1], 'b.-', label='Path of least action')
plt.legend()
plt.tight_layout() ; plt.show()

## Gas simulation

In [ ]:
dt = 0.5 ; N = 50
t_sim, x_sim = simulate_gas(dt=dt, N=N)
init_path = PerturbedPath(x_sim, N=N, sigma=1e-2) # [time, N*2]
# t_min, path, xs_min = get_path_between(init_path, steps=500, step_size=1e-3,
#                                        L_fn=lagrangian_gas, dt=dt, opt='adam')
t_min, path, xs_min = get_path_between(init_path, steps=500, step_size=1e1,
                                       L_fn=lagrangian_gas, dt=dt, opt='sgd')

In [ ]:
N = x_sim.shape[-2]
xs_before = xs_min[0].detach().numpy().reshape(-1,N,2)
xs_after = xs_min[-1].detach().numpy().reshape(-1,N,2)

k = 25
plt.figure(dpi=100)
plt.title('Ball {} horiz. velocity vs. time'.format(1 + k//2))
plt.plot((xs_before[1:] - xs_before[:-1]).reshape(-1,N*2)[...,k], '.-', label='Initial path')
plt.plot((xs_after[1:] - xs_after[:-1]).reshape(-1,N*2)[...,k], '.-', label='Minimum action')
plt.plot((x_sim[1:] - x_sim[:-1]).reshape(-1,N*2)[...,k], 'k-', label='Simulator')
plt.legend()
plt.show()

In [ ]:
xs = xs_min[0].detach().numpy().reshape(-1,N,2)
make_video(xs, path='sim.mp4', interval=30, ms=10)
mp4 = open('sim.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML('<video width=300 controls><source src="{}" type="video/mp4"></video>'.format(data_url))

In [ ]:
xs = xs_min[-1].detach().numpy().reshape(-1,N,2)
make_video(xs, path='sim.mp4', interval=30, ms=10)
mp4 = open('sim.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML('<video width=300 controls><source src="{}" type="video/mp4"></video>'.format(data_url))

## 3 body simulation

In [ ]:
dt = 0.5 ; N = 3

t_sim, x_sim = simulate_3body(dt=dt)
init_path = PerturbedPath(x_sim, N=N, sigma=2e-2) # [time, N*2]
t_min, path, xs_min = get_path_between(init_path, steps=500, step_size=1e1,
                                       L_fn=lagrangian_3body, dt=dt, opt='sgd')

In [ ]:
N = x_sim.shape[-2]
xs_before = xs_min[0].detach().numpy().reshape(-1,N,2)
xs_after = xs_min[-1].detach().numpy().reshape(-1,N,2)

plt.figure(dpi=100) ; k = 1
plt.title('Ball {} horiz. velocity vs. time'.format(1 + k//2))
plt.plot((xs_before[1:] - xs_before[:-1]).reshape(-1,N*2)[...,k], '.-', label='Initial path')
plt.plot((xs_after[1:] - xs_after[:-1]).reshape(-1,N*2)[...,k], '.-', label='Minimum action')
plt.plot((x_sim[1:] - x_sim[:-1]).reshape(-1,N*2)[...,k], 'k-', label='Simulator')
plt.legend()
plt.show()

In [ ]:
xs = xs_min[0].detach().numpy().reshape(-1,N,2)
make_video(xs, path='sim.mp4', interval=60, ms=20)
mp4 = open('sim.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML('<video width=300 controls><source src="{}" type="video/mp4"></video>'.format(data_url))

In [ ]:
xs = xs_min[-1].detach().numpy().reshape(-1,N,2)
make_video(xs, path='sim.mp4', interval=60, ms=20)
mp4 = open('sim.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML('<video width=300 controls><source src="{}" type="video/mp4"></video>'.format(data_url))

In [ ]:
# def dblpend_forces_fn(x, xdot, m1=1, m2=1, l1=1, l2=2, g=1):
#     theta1, theta2 = x
#     z1, z2 = xdot
#     c, s = np.cos(theta1-theta2), np.sin(theta1-theta2)

#     theta1dot = z1
#     z1dot = (m2*g*np.sin(theta2)*c - m2*s*(l1*z1**2*c + l2*z2**2) -
#              (m1+m2)*g*np.sin(theta1)) / l1 / (m1 + m2*s**2)
#     theta2dot = z2
#     z2dot = ((m1+m2)*(l1*z1**2*s - g*np.sin(theta2) + g*np.sin(theta1)*c) + 
#              m2*l2*z2**2*s*c) / l2 / (m1 + m2*s**2)
#     return np.asarray([z1dot, z2dot]) # theta1dot, theta2dot, 

In [ ]:
# def dblpend_forces_fn(x, xdot, m1=1, m2=1, l1=1, l2=2, g=1): # a 'd' on end of variable name means 'dot'
#     th1, th2 = x
#     th1d, th2d = xdot
#     c, s = np.cos(th1 - th2), np.sin(th1 - th2)
    
#     th1dd = (m2 * g * np.sin(th2) * c - m2 * s * (l1 * th1d ** 2 * c + l2 * th1d ** 2) -
#              (m1 + m2) * g * np.sin(th1)) / l1 / (m1 + m2 * s ** 2)
    
#     th2dd = ((m1 + m2) * (l1 * th1d ** 2 * s - g * np.sin(th2) + g * np.sin(th1) * c) +
#              m2 * l2 * th2d ** 2 * s * c) / l2 / (m1 + m2 * s ** 2)
#     return np.asarray([th1dd, th2dd])

In [ ]:
# def dblpend_forces_fn(x, m1=1, m2=1, l1=1, l2=2, g=1): # a 'd' on end of variable name means 'dot'
#     th1, th2, z1, z2 = x
#     c, s = np.cos(th1 - th2), np.sin(th1 - th2)

#     th1d = z1
#     z1d = (m2 * g * np.sin(th2) * c - m2 * s * (l1 * z1 ** 2 * c + l2 * z2 ** 2) -
#              (m1 + m2) * g * np.sin(th1)) / l1 / (m1 + m2 * s ** 2)
#     th2d = z2
#     z2d = ((m1 + m2) * (l1 * z1 ** 2 * s - g * np.sin(th2) + g * np.sin(th1) * c) +
#              m2 * l2 * z2 ** 2 * s * c) / l2 / (m1 + m2 * s ** 2)
#     return th1d, th2d, z1d, z2d